**Food Query**
SELECT DISTINCT Year,
 SQLDATE,
 Actor1CountryCode,
 Actor1Geo_Fullname,
 Actor1Geo_CountryCode,
 AvgTone,
 SOURCEURL
FROM
 `gdelt-bq.full.events`
WHERE
 (Year = 2023
   OR Year = 2024)
 AND ( LOWER(SOURCEURL) LIKE '%food%'
   OR LOWER(SOURCEURL) LIKE '%grocery%' )
 AND Actor1CountryCode = 'USA';



**Transportation Query**
SELECT
 DISTINCT Year,
 SQLDATE,
 Actor1CountryCode,
 Actor1Geo_Fullname,
 Actor1Geo_CountryCode,
 AvgTone,
 SOURCEURL
FROM
 `gdelt-bq.full.events`
WHERE
 (Year = 2023
   OR Year = 2024)
 AND ( LOWER(SOURCEURL) LIKE '%transportation%'
   OR LOWER(SOURCEURL) LIKE '%congestion-pricing%' )
 AND Actor1CountryCode = 'USA';


**Housing Query**
SELECT DISTINCT
   Year,
   ActionGeo_Lat,
   ActionGeo_Long,
   SQLDATE,
   Actor1CountryCode,
   Actor1Geo_Fullname,
   Actor1Geo_CountryCode,
   AvgTone,
   SOURCEURL
FROM
   `gdelt-bq.full.events`
WHERE
   (Year = 2023 OR Year = 2024)
   AND (
       LOWER(SOURCEURL) LIKE '%housing%'
       OR LOWER(SOURCEURL) LIKE '%home-prices%'
       OR LOWER(SOURCEURL) LIKE '%real-estate%'
       OR LOWER(SOURCEURL) LIKE '%house-prices%'
       OR LOWER(SOURCEURL) LIKE '%mortgage%'
       OR LOWER(SOURCEURL) LIKE '%property%'
   )
   AND Actor1CountryCode = 'USA';



In [ ]:
# Running this code will query a table in BigQuery and download
# Learn more here: https://cloud.google.com/bigquery/docs/visualize-jupyter

%%bigquery food --project sincere-pen-442701-r8
SELECT * FROM `sincere-pen-442701-r8.gdelt.food` #this table name was set based on the table you chose to query

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
food=pd.DataFrame(food)

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
# Convert SQLDATE to datetime and extract week number
food['week_number'] = pd.to_datetime(food['SQLDATE'], format='%Y%m%d').dt.isocalendar().week

In [ ]:
food['category'] = 'food'

In [ ]:
destination_table = "gdelt.food"  # Use only dataset and table name

# Save the DataFrame to BigQuery
to_gbq(food, destination_table=destination_table, if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 8439.24it/s]


In [ ]:
# Running this code will query a table in BigQuery and download
# the results to a Pandas DataFrame named `results`.
# Learn more here: https://cloud.google.com/bigquery/docs/visualize-jupyter

%%bigquery housing --project sincere-pen-442701-r8
SELECT * FROM `sincere-pen-442701-r8.gdelt.housing` #this table name was set based on the table you chose to query

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
housing=pd.DataFrame(housing)

In [ ]:
housing['week_number'] = pd.to_datetime(housing['SQLDATE'], format='%Y%m%d').dt.isocalendar().week
housing['category'] = 'housing'

In [ ]:
destination_table = "gdelt.food"  # Use only dataset and table name

# Save the DataFrame to BigQuery
to_gbq(food, destination_table=destination_table, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 7345.54it/s]


In [ ]:
%%bigquery transportation --project sincere-pen-442701-r8
SELECT * FROM `sincere-pen-442701-r8.gdelt.transportation` #this table name was set based on the table you chose to query

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
transportation=pd.DataFrame(transportation)

In [ ]:
transportation['week_number'] = pd.to_datetime(transportation['SQLDATE'], format='%Y%m%d').dt.isocalendar().week
transportation['category'] = 'transportation'

In [ ]:
df = pd.concat([food, housing, transportation], ignore_index=True)

In [ ]:
df['date'] = pd.to_datetime(df['SQLDATE'], format='%Y%m%d')

In [ ]:
grouped_by_week = df.groupby('week_number', group_keys=False)

In [ ]:
df['Percentile_25'] = grouped_by_week['AvgTone'].transform(lambda x: x.quantile(0.25))
df['Percentile_75'] = grouped_by_week['AvgTone'].transform(lambda x: x.quantile(0.75))

In [ ]:
gdelt = df.sort_values(by='week_number')

In [ ]:
max_commas = gdelt['Actor1Geo_Fullname'].fillna('').str.split(',').apply(len).max()

# Split the 'Actor1Geo_Fullname' by commas and expand it into multiple columns
gdelt[[f'name_{x}' for x in range(max_commas)]] = gdelt['Actor1Geo_Fullname'].fillna('').str.split(',', expand=True)

In [ ]:
gdelt=gdelt[['Year','SQLDATE', 'Actor1Geo_CountryCode', 'AvgTone','SOURCEURL','week_number','category','date','Percentile_25', 'Percentile_75','name_1']].rename(
    columns={'name_1':'state'}
)

In [ ]:
destination_table = "gdelt.df"  # Use only dataset and table name

# Save the DataFrame to BigQuery
to_gbq(gdelt, destination_table=destination_table, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 9510.89it/s]
